In [1]:
%%capture
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Load the pre-trained InceptionResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = True

# Add custom layers on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
def lr_scheduler(epoch):
    return 1e-3 / (1 + epoch)

optimizer = optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Print the model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_resnet_v2             │ ?                      │    54,336,736 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_407         │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,336,736 (207.28 MB)

 Trainable params: 54,276,192 (207.05 MB)

 Non-trainable params: 60,544 (236.50 KB)

In [10]:
import tensorflow as tf

# Define the path to your dataset
dataset_path = '/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI'

# Load the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,  # Reserve 20% of data for validation
    subset="training",
    seed=123,  # Seed for reproducibility
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=32  # Number of images to return in each batch
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,  # Reserve 20% of data for validation
    subset="validation",
    seed=123,  # Seed for reproducibility
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=32  # Number of images to return in each batch
)

# Prefetch data for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 8994 files belonging to 2 classes.
Using 7196 files for training.
Found 8994 files belonging to 2 classes.
Using 1798 files for validation.


In [11]:
history = model.fit(train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=3, 
                    batch_size=32,
                    shuffle=True)

Epoch 1/3
225/225 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.8074 - loss: 0.4279 - precision_1: 0.8202 - recall_1: 0.8259 - val_accuracy: 0.7358 - val_loss: 0.5719 - val_precision_1: 0.8674 - val_recall_1: 0.5877
Epoch 2/3
225/225 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.8310 - loss: 0.3820 - precision_1: 0.8429 - recall_1: 0.8453 - val_accuracy: 0.6869 - val_loss: 0.6631 - val_precision_1: 0.6331 - val_recall_1: 0.9630
Epoch 3/3
225/225 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.8615 - loss: 0.3259 - precision_1: 0.8680 - recall_1: 0.8802 - val_accuracy: 0.6769 - val_loss: 0.7457 - val_precision_1: 0.8230 - val_recall_1: 0.4915


In [16]:
history = model.fit(train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=30, 
                    batch_size=32,
                    shuffle=True)

Epoch 1/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.9575 - loss: 0.1180 - precision_1: 0.9588 - recall_1: 0.9630 - val_accuracy: 0.7636 - val_loss: 0.9947 - val_precision_1: 0.7592 - val_recall_1: 0.8066
Epoch 2/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.9617 - loss: 0.1098 - precision_1: 0.9658 - recall_1: 0.9633 - val_accuracy: 0.7597 - val_loss: 0.7804 - val_precision_1: 0.7362 - val_recall_1: 0.8467
Epoch 3/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 270s 1s/step - accuracy: 0.9695 - loss: 0.0804 - precision_1: 0.9687 - recall_1: 0.9753 - val_accuracy: 0.7525 - val_loss: 3.6661 - val_precision_1: 0.7114 - val_recall_1: 0.8911
Epoch 4/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.9702 - loss: 0.0770 - precision_1: 0.9693 - recall_1: 0.9756 - val_accuracy: 0.7631 - val_loss: 1.0389 - val_precision_1: 0.7439 - val_recall_1: 0.8383
Epoch 5/30
225/225 ━━━━━━━━━━━━━━━━━━━━ 265s 1s/step - accuracy: 0.9703 - loss: 0.0867 - precision_1: 0.9708 - recall_1: 0.9

KeyboardInterrupt: 

In [17]:
import tensorflow as tf

# Assume 'model' is your trained TensorFlow model
model.save('/kaggle/working/model3.h5')


In [9]:
# history = model.fit(train_dataset, 
#                     validation_data=validation_dataset, 
#                     epochs=3, 
#                     batch_size=32,
#                     shuffle=True)

Epoch 1/3
225/225 ━━━━━━━━━━━━━━━━━━━━ 260s 1s/step - accuracy: 0.7781 - loss: 0.4812 - precision_1: 0.7869 - recall_1: 0.8080 - val_accuracy: 0.4911 - val_loss: 1.0810 - val_precision_1: 0.9429 - val_recall_1: 0.0349
Epoch 2/3
 15/225 ━━━━━━━━━━━━━━━━━━━━ 3:10 908ms/step - accuracy: 0.7784 - loss: 0.5184 - precision_1: 0.7653 - recall_1: 0.8499

KeyboardInterrupt: 

In [ ]:
# results = model.evaluate(validation_dataset)

# # Print results
# print(f'Accuracy: {results[1]:.2f}')
# print(f'Recall: {results[2]:.2f}')
# print(f'Precision: {results[3]:.2f}')